In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('gtb20-21data.txt')
winter2 = pd.read_csv ('gtb21-22data.txt')
winter3 = pd.read_csv ('gtb22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
precip_days2

,station,datetime,precip(mm),wxcodes
3,GTB,2021-12-01 01:05:00,0.00,-UP
4,GTB,2021-12-01 01:08:00,0.00,-SN
5,GTB,2021-12-01 01:48:00,0.00,-SN BR
6,GTB,2021-12-01 01:56:00,T,-SN BR
7,GTB,2021-12-01 02:06:00,0.00,-SN BR
...,...,...,...,...
5252,GTB,2022-02-27 21:09:00,0.00,-UP
5253,GTB,2022-02-27 21:12:00,0.00,-SN
5254,GTB,2022-02-27 21:29:00,0.00,-UP
5259,GTB,2022-02-27 22:06:00,0.00,-UP


In [7]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [8]:
precip_days3[68:70]

,station,datetime,precip(mm),wxcodes
153,GTB,2022-12-03 18:32:00,0.00,-RA
154,GTB,2022-12-03 18:42:00,0.00,-RA


In [9]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [10]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [11]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [12]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [13]:
gtb = LocationInfo(
    'Fort Drum' 'New York', 
    latitude= 44.0453,
    longitude= -75.7953, 
    timezone='America/New_York'
)

gtb

LocationInfo(name='Fort DrumNew York', region='England', timezone='America/New_York', latitude=44.0453, longitude=-75.7953)

In [14]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(gtb.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [15]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(gtb.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [16]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(gtb.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [17]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:18:52  2020-12-01 16:25:47  2020-12-01
1   2020-12-02 07:19:57  2020-12-02 16:25:29  2020-12-02
2   2020-12-03 07:21:02  2020-12-03 16:25:13  2020-12-03
3   2020-12-04 07:22:05  2020-12-04 16:25:00  2020-12-04
4   2020-12-05 07:23:06  2020-12-05 16:24:50  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:48:12  2021-02-24 17:45:04  2021-02-24
86  2021-02-25 06:46:34  2021-02-25 17:46:24  2021-02-25
87  2021-02-26 06:44:54  2021-02-26 17:47:43  2021-02-26
88  2021-02-27 06:43:14  2021-02-27 17:49:02  2021-02-27
89  2021-02-28 06:41:33  2021-02-28 17:50:21  2021-02-28

[90 rows x 3 columns]


In [18]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:18:36  2021-12-01 16:25:52  2021-12-01
1   2021-12-02 07:19:42  2021-12-02 16:25:33  2021-12-02
2   2021-12-03 07:20:46  2021-12-03 16:25:17  2021-12-03
3   2021-12-04 07:21:49  2021-12-04 16:25:03  2021-12-04
4   2021-12-05 07:22:51  2021-12-05 16:24:52  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:48:36  2022-02-24 17:44:45  2022-02-24
86  2022-02-25 06:46:58  2022-02-25 17:46:05  2022-02-25
87  2022-02-26 06:45:18  2022-02-26 17:47:24  2022-02-26
88  2022-02-27 06:43:38  2022-02-27 17:48:43  2022-02-27
89  2022-02-28 06:41:58  2022-02-28 17:50:02  2022-02-28

[90 rows x 3 columns]


In [19]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:18:20  2022-12-01 16:25:56  2022-12-01
1   2022-12-02 07:19:26  2022-12-02 16:25:37  2022-12-02
2   2022-12-03 07:20:31  2022-12-03 16:25:20  2022-12-03
3   2022-12-04 07:21:34  2022-12-04 16:25:06  2022-12-04
4   2022-12-05 07:22:36  2022-12-05 16:24:54  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:48:59  2023-02-24 17:44:26  2023-02-24
86  2023-02-25 06:47:21  2023-02-25 17:45:45  2023-02-25
87  2023-02-26 06:45:42  2023-02-26 17:47:05  2023-02-26
88  2023-02-27 06:44:03  2023-02-27 17:48:24  2023-02-27
89  2023-02-28 06:42:22  2023-02-28 17:49:42  2023-02-28

[90 rows x 3 columns]


In [20]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [21]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GTB,2020-12-01 06:56:00,0.0001,-RA FG,2020-12-01,2020-12-01 07:18:52,2020-12-01 16:25:47
1,GTB,2020-12-01 12:56:00,0.2500,-RA BR,2020-12-01,2020-12-01 07:18:52,2020-12-01 16:25:47
2,GTB,2020-12-01 13:56:00,1.0200,-RA BR,2020-12-01,2020-12-01 07:18:52,2020-12-01 16:25:47
3,GTB,2020-12-02 06:56:00,0.0001,-UP,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29
4,GTB,2020-12-02 07:56:00,0.0001,-SN,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29
...,...,...,...,...,...,...,...
449,GTB,2021-02-26 10:56:00,0.0001,-SN,2021-02-26,2021-02-26 06:44:54,2021-02-26 17:47:43
450,GTB,2021-02-27 13:56:00,0.5100,-RA,2021-02-27,2021-02-27 06:43:14,2021-02-27 17:49:02
451,GTB,2021-02-27 15:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:43:14,2021-02-27 17:49:02
452,GTB,2021-02-27 16:56:00,0.5100,-RA,2021-02-27,2021-02-27 06:43:14,2021-02-27 17:49:02


In [22]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GTB,2021-12-01 01:56:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:18:36,2021-12-01 16:25:52
1,GTB,2021-12-01 02:56:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:18:36,2021-12-01 16:25:52
2,GTB,2021-12-01 03:56:00,0.2500,-SN,2021-12-01,2021-12-01 07:18:36,2021-12-01 16:25:52
3,GTB,2021-12-01 04:56:00,0.5100,-SN BR,2021-12-01,2021-12-01 07:18:36,2021-12-01 16:25:52
4,GTB,2021-12-01 05:56:00,0.5100,-RA,2021-12-01,2021-12-01 07:18:36,2021-12-01 16:25:52
...,...,...,...,...,...,...,...
420,GTB,2022-02-27 13:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:43:38,2022-02-27 17:48:43
421,GTB,2022-02-27 14:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:43:38,2022-02-27 17:48:43
422,GTB,2022-02-27 15:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:43:38,2022-02-27 17:48:43
423,GTB,2022-02-27 16:56:00,0.0001,-UP,2022-02-27,2022-02-27 06:43:38,2022-02-27 17:48:43


In [23]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GTB,2022-12-01 13:56:00,0.0000,-SN,2022-12-01,2022-12-01 07:18:20,2022-12-01 16:25:56
1,GTB,2022-12-01 14:56:00,0.0000,-RA,2022-12-01,2022-12-01 07:18:20,2022-12-01 16:25:56
2,GTB,2022-12-11 13:56:00,0.0000,-UP,2022-12-11,2022-12-11 07:28:16,2022-12-11 16:24:37
3,GTB,2022-12-16 00:56:00,0.0000,-SN,2022-12-16,2022-12-16 07:32:09,2022-12-16 16:25:33
4,GTB,2022-12-16 19:56:00,0.0000,-RA,2022-12-16,2022-12-16 07:32:09,2022-12-16 16:25:33
5,GTB,2022-12-17 01:56:00,0.0000,-SN BR,2022-12-17,2022-12-17 07:32:50,2022-12-17 16:25:51
6,GTB,2022-12-17 02:56:00,0.0000,-SN,2022-12-17,2022-12-17 07:32:50,2022-12-17 16:25:51
7,GTB,2022-12-17 21:56:00,0.0000,-SN,2022-12-17,2022-12-17 07:32:50,2022-12-17 16:25:51
8,GTB,2022-12-24 08:56:00,0.0000,BLSN,2022-12-24,2022-12-24 07:36:32,2022-12-24 16:29:11
9,GTB,2022-12-28 13:56:00,0.0000,-SN BR,2022-12-28,2022-12-28 07:37:47,2022-12-28 16:31:55


In [24]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
1,GTB,2020-12-01 12:56:00,0.2500,-RA BR,2020-12-01,2020-12-01 07:18:52,2020-12-01 16:25:47
2,GTB,2020-12-01 13:56:00,1.0200,-RA BR,2020-12-01,2020-12-01 07:18:52,2020-12-01 16:25:47
4,GTB,2020-12-02 07:56:00,0.0001,-SN,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29
5,GTB,2020-12-02 08:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29
6,GTB,2020-12-02 09:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29
...,...,...,...,...,...,...,...
448,GTB,2021-02-26 09:56:00,0.0001,-UP,2021-02-26,2021-02-26 06:44:54,2021-02-26 17:47:43
449,GTB,2021-02-26 10:56:00,0.0001,-SN,2021-02-26,2021-02-26 06:44:54,2021-02-26 17:47:43
450,GTB,2021-02-27 13:56:00,0.5100,-RA,2021-02-27,2021-02-27 06:43:14,2021-02-27 17:49:02
451,GTB,2021-02-27 15:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:43:14,2021-02-27 17:49:02


In [25]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [26]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
1,GTB,2020-12-01 12:56:00,0.2500,-RA BR,2020-12-01,2020-12-01 07:18:52,2020-12-01 16:25:47,2020-12-01 08:18:52,2020-12-01 15:25:47
2,GTB,2020-12-01 13:56:00,1.0200,-RA BR,2020-12-01,2020-12-01 07:18:52,2020-12-01 16:25:47,2020-12-01 08:18:52,2020-12-01 15:25:47
5,GTB,2020-12-02 08:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29,2020-12-02 08:19:57,2020-12-02 15:25:29
6,GTB,2020-12-02 09:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29,2020-12-02 08:19:57,2020-12-02 15:25:29
7,GTB,2020-12-02 10:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29,2020-12-02 08:19:57,2020-12-02 15:25:29
...,...,...,...,...,...,...,...,...,...
447,GTB,2021-02-26 08:56:00,0.0001,-SN,2021-02-26,2021-02-26 06:44:54,2021-02-26 17:47:43,2021-02-26 07:44:54,2021-02-26 16:47:43
448,GTB,2021-02-26 09:56:00,0.0001,-UP,2021-02-26,2021-02-26 06:44:54,2021-02-26 17:47:43,2021-02-26 07:44:54,2021-02-26 16:47:43
449,GTB,2021-02-26 10:56:00,0.0001,-SN,2021-02-26,2021-02-26 06:44:54,2021-02-26 17:47:43,2021-02-26 07:44:54,2021-02-26 16:47:43
450,GTB,2021-02-27 13:56:00,0.5100,-RA,2021-02-27,2021-02-27 06:43:14,2021-02-27 17:49:02,2021-02-27 07:43:14,2021-02-27 16:49:02


In [27]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [28]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [29]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.07466405228758169
2021-2022: 0.1799268656716418
2022-2023: 0.031675


In [30]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GTB,2020-12-01 06:56:00,0.0001,-RA FG,2020-12-01,2020-12-01 07:18:52,2020-12-01 16:25:47
3,GTB,2020-12-02 06:56:00,0.0001,-UP,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29
13,GTB,2020-12-02 16:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29
14,GTB,2020-12-02 17:56:00,0.0001,-SN,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29
15,GTB,2020-12-02 18:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:19:57,2020-12-02 16:25:29
...,...,...,...,...,...,...,...
440,GTB,2021-02-24 21:56:00,0.2500,-RA,2021-02-24,2021-02-24 06:48:12,2021-02-24 17:45:04
443,GTB,2021-02-26 03:56:00,0.0001,-SN,2021-02-26,2021-02-26 06:44:54,2021-02-26 17:47:43
444,GTB,2021-02-26 04:56:00,0.0001,-SN BR,2021-02-26,2021-02-26 06:44:54,2021-02-26 17:47:43
445,GTB,2021-02-26 05:56:00,0.0001,-SN BR,2021-02-26,2021-02-26 06:44:54,2021-02-26 17:47:43


In [31]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_19048\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [32]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [33]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [34]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.15755039370078744
2021-2022: 0.2792391666666667
2022-2023: 0.1977888888888889


In [35]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('gtb.csv', index=False)